In [2]:
from time import time
import warnings
import numpy as np
import pickle as pkl
from parser_all import parse
from qiskit import Aer
from qiskit.utils.quantum_instance import QuantumInstance
from generate_qubos import solve_classically, arr_to_str
# from qiskit.algorithms.optimizers import COBYLA, SLSQP
from classical_optimizers import NLOPT_Optimizer
from QAOAEx import QAOACustom, convert_from_fourier_point, convert_to_fourier_point
from QAOA_methods import (CustomQAOA,
                         generate_points,
                         get_costs,
                         find_all_ground_states,
                         count_coupling_terms,
                         interp_point,
                         construct_initial_state,
                         n_qbit_mixer)
from QAOAEx import convert_from_fourier_point, convert_to_fourier_point

In [124]:
with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(3, 3, 3), 'rb') as f:
    qubo, max_coeff, operator, offset, routes = pkl.load(f)
from qiskit_optimization import QuadraticProgram
qubo = QuadraticProgram()
qubo.from_ising(operator)
x_s, opt_value = find_all_ground_states(qubo)

optimal function value: -232.5
optimal value: [0. 1. 0. 0. 1. 0. 1. 0. 0.]
status: SUCCESS


In [125]:
optimizer = NLOPT_Optimizer(method = "LN_COBYLA", result_message=False)

In [126]:
backend = Aer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend = backend)

In [127]:
initial_state = construct_initial_state(no_routes = 3, no_cars = 3)
mixer = n_qbit_mixer(initial_state)

In [128]:
mixer.draw()

┌────────────┐┌─────────┐┌─────────┐
q_0: ┤0           ├┤ RZ(2*t) ├┤0        ├
     │            │├─────────┤│         │
q_1: ┤1           ├┤ RZ(2*t) ├┤1        ├
     │            │├─────────┤│         │
q_2: ┤2           ├┤ RZ(2*t) ├┤2        ├
     │            │├─────────┤│         │
q_3: ┤3           ├┤ RZ(2*t) ├┤3        ├
     │            │├─────────┤│         │
q_4: ┤4 U_init_dg ├┤ RZ(2*t) ├┤4 U_init ├
     │            │├─────────┤│         │
q_5: ┤5           ├┤ RZ(2*t) ├┤5        ├
     │            │├─────────┤│         │
q_6: ┤6           ├┤ RZ(2*t) ├┤6        ├
     │            │├─────────┤│         │
q_7: ┤7           ├┤ RZ(2*t) ├┤7        ├
     │            │├─────────┤│         │
q_8: ┤8           ├┤ RZ(2*t) ├┤8        ├
     └────────────┘└─────────┘└─────────┘

In [129]:
p = 2
fourier_point = [-0.07992929, -0.07786024,  0.45417582, -0.13951623]

In [224]:
qaoa = QAOACustom(optimizer = optimizer, reps = p, initial_state = initial_state, mixer = mixer, initial_point = fourier_point, include_custom= True, quantum_instance = quantum_instance)
# qaoa.set_parameterise_point_for_energy_evaluation(convert_from_fourier_point)
# qaoa_results = qaoa.compute_minimum_eigenvalue(operator= operator)

In [225]:
from qiskit.circuit import Parameter
parameters = [ Parameter('u_{}'.format(i)) for i in range(p) ] + [ Parameter('v_{}'.format(i)) for i in range(p) ]
circ = qaoa.construct_circuit(parameter=parameters, operator = operator)

In [226]:
circ[0].draw(fold=150)

┌────────────┐                     ┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌──────────────┐┌───┐      ┌───┐      ┌──────────────┐                »
q_0: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(56.0*u_0) ├┤ X ├┤ X ├┤ RZ(53.0*u_0) ├┤ X ├──────┤ X ├──────┤ RZ(10.0*u_0) ├────────────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└──────────────┘└─┬─┘└─┬─┘└──────────────┘└─┬─┘      └─┬─┘      └────┬───┬─────┘┌──────────────┐»
q_1: ──────────────┤ RY(π/2) ├──■──┤ X ├───────■────────────────────■────┼────────────────────┼──────────┼─────────────┤ X ├──────┤ RZ(51.0*u_0) ├»
                   └─────────┘┌─┴─┐└───┘                                 │                    │          │             └─┬─┘      └──────────────┘»
q_2: ─────────────────────────┤ X ├──────────────────────────────────────■────────────────────■──────────┼───────────────■────────────────────────»
     ┌────────────┐           └───┘     ┌───┐                                                            │                                        »
q_3: ┤ RY(1.9106) ├─────■────────────■──┤ X ├────────────────────────────────────────────────────────────■────────────────────────────────────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘                                                                                                     »
q_4: ──────────────┤ RY(π/2) ├──■──┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────»
                   └─────────┘┌─┴─┐└───┘                                                                                                          »
q_5: ─────────────────────────┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ┌────────────┐           └───┘     ┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌──────────────┐┌───┐┌───────────────┐                                »
q_6: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(41.0*u_0) ├┤ X ├┤ X ├┤ RZ(41.0*u_0) ├┤ X ├┤ RZ(-42.0*u_0) ├────────────────────────────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└──────────────┘└─┬─┘└─┬─┘└──────────────┘└─┬─┘└─────┬───┬─────┘┌──────────────┐     ┌───┐      »
q_7: ──────────────┤ RY(π/2) ├──■──┤ X ├───────■────────────────────■────┼────────────────────┼────────┤ X ├──────┤ RZ(51.0*u_0) ├─────┤ X ├──────»
                   └─────────┘┌─┴─┐└───┘                                 │                    │        └─┬─┘      └──────────────┘     └─┬─┘      »
q_8: ─────────────────────────┤ X ├──────────────────────────────────────■────────────────────■──────────■───────────────────────────────■────────»
                              └───┘                                                                                                               »
«           ┌───┐           ┌───┐┌─────────────┐               ┌───┐     ┌───┐┌──────────────┐                               ┌───┐┌────────────────┐»
«q_0: ──────┤ X ├───────────┤ X ├┤ RZ(4.0*u_0) ├───────────────┤ X ├─────┤ X ├┤ RZ(10.0*u_0) ├───────────────────────────────┤ X ├┤ RZ(-109.0*u_0) ├»
«           └─┬─┘      ┌───┐└─┬─┘└────┬───┬────┘┌─────────────┐└─┬─┘┌───┐└─┬─┘└────┬───┬─────┘┌─────────────┐                └─┬─┘└─────┬───┬──────┘»
«q_1: ────────┼────────┤ X ├──┼───────┤ X ├─────┤ RZ(9.0*u_0) ├──┼──┤ X ├──┼───────┤ X ├──────┤ RZ(3.0*u_0) ├──────────────────┼────────┤ X ├───────»
«             │        └─┬─┘  │       └─┬─┘     └─────────────┘  │  └─┬─┘  │       └─┬─┘      └────┬───┬────┘┌──────────────┐  │        └─┬─┘       »
«q_2: ────────┼──────────■────┼─────────┼────────────────────────┼────┼────┼─────────┼─────────────┤ X ├─────┤ RZ(11.0*u_0) ├──┼──────────┼─────────»
«             │               │         │                        │    │    │         │             └─┬─┘     └──────────────┘  │          │         »
«q_3: ────────■───────────────┼─────────■────────────────────────┼────■────┼─────────┼───────────────■─────────────────────────┼──────────┼────

In [227]:
assigned_circ = circ[0].assign_parameters( {  parameter: i for (i, parameter) in enumerate(parameters) })

In [228]:
assigned_circ.draw(fold=150)

┌────────────┐                     ┌───┐┌───┐┌───────┐┌───┐┌───┐┌───────┐┌───┐  ┌───┐  ┌───────┐           ┌───┐       ┌───┐┌───────┐         »
q_0: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(0) ├┤ X ├┤ X ├┤ RZ(0) ├┤ X ├──┤ X ├──┤ RZ(0) ├───────────┤ X ├───────┤ X ├┤ RZ(0) ├─────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└───────┘└─┬─┘└─┬─┘└───────┘└─┬─┘  └─┬─┘  └─┬───┬─┘┌───────┐  └─┬─┘  ┌───┐└─┬─┘└─┬───┬─┘┌───────┐»
q_1: ──────────────┤ RY(π/2) ├──■──┤ X ├───────■─────────────■────┼─────────────┼──────┼──────┤ X ├──┤ RZ(0) ├────┼────┤ X ├──┼────┤ X ├──┤ RZ(0) ├»
                   └─────────┘┌─┴─┐└───┘                          │             │      │      └─┬─┘  └───────┘    │    └─┬─┘  │    └─┬─┘  └───────┘»
q_2: ─────────────────────────┤ X ├───────────────────────────────■─────────────■──────┼────────■─────────────────┼──────■────┼──────┼─────────────»
     ┌────────────┐           └───┘     ┌───┐                                          │                          │           │      │             »
q_3: ┤ RY(1.9106) ├─────■────────────■──┤ X ├──────────────────────────────────────────■──────────────────────────■───────────┼──────■─────────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘                                                                                 │                    »
q_4: ──────────────┤ RY(π/2) ├──■──┤ X ├──────────────────────────────────────────────────────────────────────────────────────■────────────────────»
                   └─────────┘┌─┴─┐└───┘                                                                                                           »
q_5: ─────────────────────────┤ X ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ┌────────────┐           └───┘     ┌───┐┌───┐┌───────┐┌───┐┌───┐┌───────┐┌───┐┌───────┐                                                       »
q_6: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(0) ├┤ X ├┤ X ├┤ RZ(0) ├┤ X ├┤ RZ(0) ├───────────────────────────────────────────────────────»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└───────┘└─┬─┘└─┬─┘└───────┘└─┬─┘└─┬───┬─┘┌───────┐  ┌───┐  ┌───────┐                            »
q_7: ──────────────┤ RY(π/2) ├──■──┤ X ├───────■─────────────■────┼─────────────┼────┤ X ├──┤ RZ(0) ├──┤ X ├──┤ RZ(0) ├────────────────────────────»
                   └─────────┘┌─┴─┐└───┘                          │             │    └─┬─┘  └───────┘  └─┬─┘  ├───────┤                            »
q_8: ─────────────────────────┤ X ├───────────────────────────────■─────────────■──────■─────────────────■────┤ RZ(0) ├────────────────────────────»
                              └───┘                                                                           └───────┘                            »
«     ┌───┐     ┌───┐┌───────┐                  ┌───┐┌───────┐                                                                                    »
«q_0: ┤ X ├─────┤ X ├┤ RZ(0) ├──────────────────┤ X ├┤ RZ(0) ├────────────────────────────────────────────────────────────────────────────────────»
«     └─┬─┘┌───┐└─┬─┘└─┬───┬─┘┌───────┐         └─┬─┘└─┬───┬─┘     ┌───┐┌───────┐         ┌───┐┌───────┐                                          »
«q_1: ──┼──┤ X ├──┼────┤ X ├──┤ RZ(0) ├───────────┼────┤ X ├───────┤ X ├┤ RZ(0) ├─────────┤ X ├┤ RZ(0) ├──────────────────────────────────────────»
«       │  └─┬─┘  │    └─┬─┘  └─┬───┬─┘┌───────┐  │    └─┬─┘  ┌───┐└─┬─┘└─┬───┬─┘┌───────┐└─┬─┘└─┬───┬─┘┌───┐┌───────┐         ┌───┐┌───────┐     »
«q_2: ──┼────┼────┼──────┼──────┤ X ├──┤ RZ(0) ├──┼──────┼────┤ X ├──┼────┤ X ├──┤ RZ(0) ├──┼────┤ X ├──┤ X ├┤ RZ(0) ├─────────┤ X ├┤ RZ(0) ├─────»
«       │    │    │      │      └─┬─┘  └───────┘  │      │    └─┬─┘  │    └─┬─┘  └───────┘  │    └─┬─┘  └─┬─┘└─┬───┬─┘┌───────┐└─┬─┘└─┬───┬─┘┌───┐»
«q_3: ──┼────■────┼──────┼────────■───────────────┼──────┼──────■────┼──────┼───────────────┼──────┼──────┼────┤ X ├──┤ RZ(0) ├──┼────┤ X 

In [229]:
from qiskit import execute

In [230]:
result = execute(assigned_circ, backend=backend).result()

In [231]:
from qiskit.quantum_info import Statevector
sv = result.get_statevector()

In [232]:
probs = qaoa._eigenvector_to_solutions(sv, qubo)

In [233]:
probs

[('100100100', -221.5, 0.03351866794508984),
 ('010100100', -222.5, 0.04667021698380534),
 ('001100100', -220.5, 0.026126791586241857),
 ('111100100', 90.5, 0.0010020848965117102),
 ('100010100', -231.5, 0.035023800799058334),
 ('010010100', -232.5, 0.0521108592072566),
 ('001010100', -230.5, 0.02095222112460739),
 ('111010100', 56.5, 0.0007501839956530125),
 ('100001100', -222.5, 0.024232419204597788),
 ('010001100', -223.5, 0.037090423805298416),
 ('001001100', -217.5, 0.039615849866850135),
 ('111001100', 93.5, 4.511661823600419e-05),
 ('100111100', 119.5, 0.000990992401767326),
 ('010111100', 114.5, 0.0014047945843574199),
 ('001111100', 128.5, 5.705474907266227e-05),
 ('111111100', 491.5, 8.08928028970272e-06),
 ('100100010', -211.5, 0.04080310104726245),
 ('010100010', -212.5, 0.05681280600432793),
 ('001100010', -210.5, 0.03180478767475478),
 ('111100010', 100.5, 0.0012198626555591577),
 ('100010010', -221.5, 0.04263533638640654),
 ('010010010', -222.5, 0.06343583394711955),
 ('

In [234]:
total = sum( [item[1]* item[2] for item in probs] )

In [235]:
total

-210.58647555747623

In [240]:
sv_2 = qaoa.calculate_statevector_at_point(operator = operator, point = convert_from_fourier_point([0,1,2,3],4))
sv_2
probs_2 = qaoa._eigenvector_to_solutions(sv_2, qubo)
total_2 = sum( [item[1] * item[2] for item in probs_2] )
total_2

-13.46697657541528

In [242]:
qaoa.set_parameterise_point_for_energy_evaluation(convert_from_fourier_point)
fval = qaoa.evaluate_energy_at_point(operator = operator, point = [0,1,2,3])
fval

-13.466976575415226

In [245]:
qaoa.reset_reps(4)

In [247]:
e = qaoa.evaluate_energy_at_point(operator = operator, point = [-0.0639487, -0.07643202, 0.0395166, -0.03228677, 0.49944737, 0.07051154, 0.08479134, -0.12322544])

In [253]:
state = qaoa.calculate_statevector_at_point(operator = operator, point = convert_from_fourier_point([-0.0639487, -0.07643202, 0.0395166, -0.03228677, 0.49944737, 0.07051154, 0.08479134, -0.12322544], 8))

In [258]:
probs_2 = qaoa._eigenvector_to_solutions(state, qubo)
sorted(probs_2, key = lambda x: x[2], reverse= True)

0.9999944403978153